In [5]:
#to create R kernel:
##get R
#!module load R
#!conda install -c conda-forge r-base -y 
## to create notebook R kernel
##might need to run that one from the consol if get "EnvironmentNotWritableError"
#!conda install -c conda-forge r-irkernel -y 
## to run R code in python
#!conda install -c r rpy2 -y

##run from R:
#install.packages(c('crayon', 'pbdZMQ', 'devtools','repr', 'IRdisplay'))
#install.packages('IRkernel')
#IRkernel::installspec(user = FALSE)

In [2]:
import rpy2.robjects.packages as packages
import rpy2.robjects.lib.ggplot2 as ggplot2
import rpy2.robjects as ro

ModuleNotFoundError: No module named 'rpy2'

In [56]:
import plotnine
import os
import pandas as pd
import numpy as np
from plotnine import *
pathresults = "../results/uqrMaster2102_starchDynamic/"
lateDry = "10to18to25to25dry10871/" 
earlyDry = "10to11to18to25dry10870/" 
baseline = "10to18to25to25wet10869/" 
data_file_delimiter = ","

def getVal(directory1,filename, factor):
    file_dir = pathresults+directory1
    largest_column_count = 0
    with open(file_dir+filename, 'r') as temp_f:
        for line in temp_f:
            last_line = line
            column_count = len(last_line.split(data_file_delimiter)) + 1
            if column_count > largest_column_count:
                largest_column_count = column_count
    column_names = [i for i in range(0, largest_column_count)]
    val_ = pd.read_csv(file_dir+filename, header=None, delimiter=data_file_delimiter, names=column_names)
    
    time = pd.read_csv(file_dir+"time.txt", header=None, delimiter=data_file_delimiter)
    
    val_sum= val_.sum(axis=1)
    val_cumsum = np.cumsum(val_sum*factor)
    val_cumsum = pd.DataFrame(val_cumsum, columns = ['cumsum'])
    val_cumsum['time'] = time.iloc[:,0]
    val_cumsum['sum'] = val_sum
    return val_cumsum

def getValAll(valName,fileName, multFactor):
    valearlyDry = getVal(earlyDry,fileName, multFactor )
    valearlyDry['scenario'] = "drySpell"
    valearlyDry['growth'] = "a) 11-18d"
    vallateDry = getVal(lateDry,fileName, multFactor )
    vallateDry['scenario'] = "drySpell"
    vallateDry['growth'] = "b) 18-25d"
    valebaseline = getVal(baseline,fileName , multFactor)
    valebaseline['scenario'] = "baseline"
    valebaseline['growth'] = "b) 18-25d"

    valAll =pd.concat([valearlyDry,vallateDry,valebaseline])
    valAll['var'] = valName
    return(valAll)
def ifelse(x, list_of_checks, yes_label, no_label):
    print(x, list_of_checks)
    if x == list_of_checks:
        res = yes_label
    else: 
        res = no_label
    return(res)

In [46]:
trans = getValAll("Transpiration","transrate.txt", 1)
Ag = getValAll("Ag","AgPhl.txt", 1/24)
transAg = pd.concat([trans,Ag])

In [48]:
transAg_ = transAg

#cumsum_cm3 * (g/cm3) * (mol/g) * (mmol/mol) * ratiotrans(for visualisation)
ratioTrans = 250 
transAg_.loc[transAg_['var'] == "Transpiration",'cumsum'] *= 1000*(1/18.01528)/ratioTrans
#cumsum_cm3 * (mg/cm3) * (mmol/mg)  * scalingFactor
transAg_.loc[transAg['growth']==np.unique(transAg['growth'])[0],'growth'] = "a) early dry spell"
transAg_.loc[transAg['growth']!=np.unique(transAg['growth'])[1],'growth'] ="b) late dry spell"

In [62]:
from plotnine import options
options.figure_size = (35, 15)
(
    ggplot(data=transAg_)
    +aes(x='time', y='cumsum',col ='var',linetype ='scenario',
                                size='scenario',alpha='scenario')
    +geom_line()
    +scale_size_manual(breaks=["drySpell","baseline"],name="",
                    values = [1.5,0.5], drop = False)
    +scale_alpha_manual(breaks=["drySpell","baseline"],name="",
                     values = [0.3,1], drop = False)
    +scale_x_continuous(breaks=range(5,28,2))
    #+scale_y_continuous(sec_axis = sec_axis(~.*ratioTrans, name = "cumulative transpiration (mmol)"))
    +xlab('day of growth (d)')
    +facet_wrap("growth",scales = "fixed", ncol = 1)
    #+ylab(bquote(A["g"]~" (mmol)"))
    #+geom_vline(data=filter(transAg_, growth==unique(transAg_['growth'])[1]),xintercept=11, colour="black") + 
  #geom_vline(data=filter(transAg_, growth==unique(transAg_['growth'])[1]), xintercept=18, colour="black") + 
  #geom_vline(data=filter(transAg_, growth==unique(transAg_['growth'])[2]),xintercept=18, colour="black") + 
  #geom_vline(data=filter(transAg_, growth==unique(transAg_['growth'])[2]), xintercept=25, colour="black") + 
  +scale_color_manual(breaks=["Transpiration","Ag"],name="",
                     #labels=vlabeller,#c("cumulative transpiration","cumulative Ag"),
                     values = ["#8DA0CB","#FC8D62"],
                     guide = "none")+  
  labs(colour = "", linetype ="")+
  theme(legend_text = element_text( size=15),
        legend_title = element_text( size=15),
        legend_position=[0.2,0.3],
        #legend_direction = "horizontal",
        legend_box = "vertical",
        panel_grid_minor = element_line(colour="white"),
        panel_grid_major = element_line(colour="white"),
        panel_background = element_rect(fill="white", color="black"),
        strip_text_y =element_text( size=15) ,
        axis_text = element_text( size=17),
        axis_title_x = element_text( size=21),
        axis_title_y_left = element_text( size=21,color = "#FC8D62"),
        axis_title_y_right = element_text( size=21,color = "#8DA0CB"),
        axis_line_y_left = element_line(color = "#FC8D62"),
        axis_ticks_y_left = element_line(color = "#FC8D62"),
        axis_text_y_left = element_text(color = "#FC8D62"),
        axis_line_y_right = element_line(color = "#8DA0CB"),
        axis_ticks_y_right = element_line(color = "#8DA0CB"),
        axis_text_y_right = element_text(color = "#8DA0CB"),
        strip_background = element_rect(fill="white"),
        strip_text = element_text(size = 21, hjust = 0,face ="bold"),
        strip_placement = "inside") 
)

PlotnineError: 'No such themeable element axis_title_y_left'

In [ ]:
from plotnine import options
options.figure_size = (35, 15)
(
    ggplot(df3)  # What data to use
    + aes(x='depth', y= "RLD" ,  group="scenario",  color="scenario")  # What variable to use
    + geom_line()  # Geometric object to use for drawing
    + xlab("depth")
    + coord_flip()
    + facet_wrap("phenotype")
    + ylab("RLD")
    + scale_fill_manual(name="",
                     values = ["#FC8D62","#8DA0CB"])
    
    +theme(legend_text = element_text( size=25),
        legend_title = element_text( size=25),
        panel_border = element_rect(color = "black", fill = "white", size = 1), 
        legend_position= [0.5,-0.1],
        #legend.direction = "horizontal",
        legend_box = "vertical",
        panel_grid_minor = element_line(colour="white"),
        panel_grid_major = element_line(colour="white"),
        panel_background = element_rect(fill="white", color="black"),
        strip_text_y =element_text( size=25) ,
        axis_text_x =element_text(angle = 90, size=20) ,
        axis_text = element_text( size=20),
        axis_title = element_text( size=25),
        strip_background = element_rect(fill="white"),
        strip_text = element_text(size = 25, hjust = 0,face ="bold")) 
)

In [ ]:
var.lab3 = c("Transpiration" = "cumulative transpiration", 
             "Ag" = bquote("cumulative"~A["g"]))

vlabeller <- function (variable, value) {
  return(var.lab3[value])
}


In [16]:
def getValAll(path, trans):
    time = getTrans(path+"time.txt")
    transsum= trans.sum(axis=1)
    transcumsum = np.cumsum(transsum*(1/24))
    transcumsum = pd.DataFrame(transcumsum, columns = ['cumsum'])
    transcumsum['time'] = time.iloc[:,0]
    return transcumsum

In [19]:
def GetATransform(path,name):
    myObject = getTrans(path+name)
    myObject = TransFormTrans(path, myObject)
    return myObject

In [20]:
transOD = GetATransform("6to28to28dry/","trans.txt")
transOW = GetATransform("6to28to28wet/","trans.txt")
transYW = GetATransform("6to16to28wet/","trans.txt")
transYD = GetATransform("6to16to28dry/","trans.txt")

../results/uqrMaster/6to28to28dry/trans.txt
50
../results/uqrMaster/6to28to28dry/time.txt
2
../results/uqrMaster/6to28to28wet/trans.txt
53
../results/uqrMaster/6to28to28wet/time.txt
2
../results/uqrMaster/6to16to28wet/trans.txt
60
../results/uqrMaster/6to16to28wet/time.txt
2
../results/uqrMaster/6to16to28dry/trans.txt
41
../results/uqrMaster/6to16to28dry/time.txt
2


In [18]:
transOD

,cumsum,time
0,0.007100,6.000000
1,0.015713,6.041667
2,0.025880,6.083333
3,0.037502,6.125000
4,0.050318,6.166667
...,...,...
446,79.669149,24.583333
447,79.690636,24.625000
448,79.735284,24.666667
449,79.751981,24.708333
